In [245]:
import hashlib
import time
import ecdsa
import random
import string

### Defines a wallet with a public and private key and balance

In [275]:
class Wallet:
    def __init__(self, name):
        # Generate a random private and public key pair for the wallet using secp256k1 curve
        self.name = name
        self.private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1).to_string().hex()
        self.public_key = hashlib.sha256(self.private_key.encode()).hexdigest()
        self.balance = 0

    def add_balance(self, amount):
        self.balance += amount
        return self
    
    def display_all_wallet(self):
        print(f"Wallet {self.name}:")
        print(f"  Public Key: {self.public_key}")
        print(f"  Private Key: {self.private_key}")
        print(f"  Balance: {self.balance} units")

    def display_public_key(self):
        print(f"Public Key: {self.public_key}")

    def display_private_key(self):
        print(f"Private Key: {self.private_key}")

### Defines the transaction sender, recipient, amount, time, id

In [276]:
class Transaction:
    def __init__(self, sender, recipient, amount):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.timestamp = time.time()
        self.transaction_id = self.calculate_transaction_id()

    def calculate_transaction_id(self):
        # Hash the transaction details to create a unique identifier
        data = f"{self.sender.public_key if self.sender else 'None'}{self.recipient.public_key}{self.amount}{self.timestamp}"
        return hashlib.sha256(data.encode()).hexdigest()

    def end_transaction(self):
        if self.sender and self.sender.balance < self.amount:
            raise ValueError("Insufficient funds for the transaction.")

        # Add the amount to the recipient's balance
        self.recipient.balance += self.amount


    def display_transaction(self):
        formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(self.timestamp))
        print("Transaction:")
        print(f"  Sender Public Key: {self.sender.public_key if self.sender else 'None'}")
        print(f"  Recipient Public Key: {self.recipient.public_key}")
        print(f"  Amount: {self.amount}")
        print(f"  Timestamp: {formatted_timestamp}")
        print(f"  Transaction ID: {self.transaction_id}")

### Defines the Block which has a number, timestamp, transaction data, previous hash and its hash

In [277]:
class Block:
    def __init__(self, block_number, previous_hash, transactions):
        self.block_number = block_number
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data = (
            str(self.block_number)
            + str(self.timestamp)
            + str([transaction.transaction_id for transaction in self.transactions])
            + str(self.previous_hash)
        )
        return hashlib.sha256(data.encode()).hexdigest()

### Defines the blockchain Creates the first block when called and then has a function to add_blocks (in a real blockchain the Blockchain.chain would be immutable but in this case it isn't)

In [278]:
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.current_transactions = []

    def create_genesis_block(self):
        # Create the first block in the chain (Genesis Block)
        return Block(0, "0", [])

    def mine_block(self, miner_wallet, miner_reward):
        # Create a new block with the pending transactions
        new_block = Block(len(self.chain), self.chain[-1].hash, self.current_transactions)
        # Reset the list of pending transactions
        self.current_transactions = []

        # Reward the miner for mining a new block
        miner_transaction = Transaction(None, miner_wallet, miner_reward)
        miner_transaction.end_transaction()

        # Execute all transactions in the mined block
        for transaction in new_block.transactions:
            transaction.end_transaction()

        # Add the new block to the blockchain
        self.chain.append(new_block)

    def add_transaction(self, transaction):
        # Ensure that the transaction is valid before adding it to the blockchain
        if not isinstance(transaction, Transaction):
            raise ValueError("Invalid transaction. Must be an instance of the Transaction class.")

        if transaction.sender:
            if transaction.sender.balance < transaction.amount:
                raise ValueError("Insufficient funds for the transaction.")

            # Deduct the amount from the sender's balance
            transaction.sender.balance -= transaction.amount

        # Add the transaction to the list of current transactions
        self.current_transactions.append(transaction)

    def display_chain(self):
        for block in self.chain:
            formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(block.timestamp))
            transaction_ids = [transaction.transaction_id for transaction in block.transactions]
            print(f"Block #{block.block_number} | Hash: {block.hash}")
            print(f"Timestamp: {formatted_timestamp}")
            print(f"Transaction IDs: {transaction_ids}")
            print("--------------")

### Mining

--------------
1. Transaction: Transfer 10 BTC from Alice to Bob
2. Transaction: Transfer 5 BTC from Bob to Charlie
3. Transaction: Transfer 8 BTC from Charlie to Alice
--------------

In [279]:
if __name__ == "__main__":
    # Create wallets
    alice_wallet = Wallet("Alice")
    bob_wallet = Wallet("Bob")
    charlie_wallet = Wallet("Charlie")
    miner_wallet = Wallet("Miner")

    # Initial balances
    alice_wallet.add_balance(100)
    bob_wallet.add_balance(50)
    charlie_wallet.add_balance(30)

# Display initial balances
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: fd147a9f01c70dccbef40a31da78d34d4af2e6fda5f437c05b121d0be9cda428
  Private Key: 202ee6190a9f3aa40c3d4792a36d1c14bb6b3b9d7ff21788514809ab7c11d281
  Balance: 100 units
Wallet Bob:
  Public Key: 8e412a0238f1c4a0fd877bee6395e20ca9e1094ea4214f4c2e13ab5af9870e06
  Private Key: 423227a321216ed9ed326012e9053c3c9d2746e7d573dd7211a7a98995a57ded
  Balance: 50 units
Wallet Charlie:
  Public Key: 74db44b942863ec16933ef2d01237a3c91352d1f965901e59021265359cad2a9
  Private Key: 468e953f6241315da3aeac79ad4a9639604350cd362f37aa0e2af30b0c36c354
  Balance: 30 units


In [280]:
# Create a blockchain
my_blockchain = Blockchain()

In [281]:
my_blockchain.display_chain() # Should only display the genesis block

Block #0 | Hash: 15ae31d2fcd575154151a993aad4b71b1ab95a8cc8b64ad08817a919fb9b2112
Timestamp: 12/05/2023 02:25:35
Transaction IDs: []
--------------


### Cria as transações e as executa (ainda não terminaram)

In [282]:
# Create transactions
transaction1 = Transaction(alice_wallet, bob_wallet, 10)
transaction2 = Transaction(bob_wallet, charlie_wallet, 5)
transaction3 = Transaction(charlie_wallet, alice_wallet, 8)

# Add transactions to the blockchain
my_blockchain.add_transaction(transaction1)
my_blockchain.add_transaction(transaction2)
my_blockchain.add_transaction(transaction3)

### Balanço Inicial das 3 Carteiras

In [283]:
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: fd147a9f01c70dccbef40a31da78d34d4af2e6fda5f437c05b121d0be9cda428
  Private Key: 202ee6190a9f3aa40c3d4792a36d1c14bb6b3b9d7ff21788514809ab7c11d281
  Balance: 90 units
Wallet Bob:
  Public Key: 8e412a0238f1c4a0fd877bee6395e20ca9e1094ea4214f4c2e13ab5af9870e06
  Private Key: 423227a321216ed9ed326012e9053c3c9d2746e7d573dd7211a7a98995a57ded
  Balance: 45 units
Wallet Charlie:
  Public Key: 74db44b942863ec16933ef2d01237a3c91352d1f965901e59021265359cad2a9
  Private Key: 468e953f6241315da3aeac79ad4a9639604350cd362f37aa0e2af30b0c36c354
  Balance: 22 units


### Publico Todos podem ver

In [284]:
transaction1.display_transaction()
transaction2.display_transaction()
transaction3.display_transaction()

Transaction:
  Sender Public Key: fd147a9f01c70dccbef40a31da78d34d4af2e6fda5f437c05b121d0be9cda428
  Recipient Public Key: 8e412a0238f1c4a0fd877bee6395e20ca9e1094ea4214f4c2e13ab5af9870e06
  Amount: 10
  Timestamp: 12/05/2023 02:25:39
  Transaction ID: 234a500d470848753650663fd9d30cc1790233d12d6873d00b24c68cb9fa2d47
Transaction:
  Sender Public Key: 8e412a0238f1c4a0fd877bee6395e20ca9e1094ea4214f4c2e13ab5af9870e06
  Recipient Public Key: 74db44b942863ec16933ef2d01237a3c91352d1f965901e59021265359cad2a9
  Amount: 5
  Timestamp: 12/05/2023 02:25:39
  Transaction ID: 0ab5ee3983cb536a39cc11e18590863356b7b1ebd9aab2d3985b9a5398038b65
Transaction:
  Sender Public Key: 74db44b942863ec16933ef2d01237a3c91352d1f965901e59021265359cad2a9
  Recipient Public Key: fd147a9f01c70dccbef40a31da78d34d4af2e6fda5f437c05b121d0be9cda428
  Amount: 8
  Timestamp: 12/05/2023 02:25:39
  Transaction ID: 59da990c62939068c287564700f49a7e75c87038ac201fa653e34f4423c442f3


### Bloco Minerado e final da transação

In [285]:
# Mine a block with a reward for the miner
miner_reward = 5
my_blockchain.mine_block(miner_wallet, miner_reward)

# Display the blockchain
my_blockchain.display_chain()

Block #0 | Hash: 15ae31d2fcd575154151a993aad4b71b1ab95a8cc8b64ad08817a919fb9b2112
Timestamp: 12/05/2023 02:25:35
Transaction IDs: []
--------------
Block #1 | Hash: d588d6767596d308769a3d0dc54426af97300c5ba36129ea76be320aca5fbbc2
Timestamp: 12/05/2023 02:25:46
Transaction IDs: ['234a500d470848753650663fd9d30cc1790233d12d6873d00b24c68cb9fa2d47', '0ab5ee3983cb536a39cc11e18590863356b7b1ebd9aab2d3985b9a5398038b65', '59da990c62939068c287564700f49a7e75c87038ac201fa653e34f4423c442f3']
--------------


### Recomepensa do miner

In [286]:
miner_wallet.display_all_wallet()

Wallet Miner:
  Public Key: 41dc194e498402b206067c5480d0465514fbd11c9f4834e9762ae7419c1e6a1e
  Private Key: 8ef55be413558f6208d1d8d37adfe2f14df877fd514f1f68748403c45f4119e7
  Balance: 5 units


### Resultado após Finalização das Transações

In [287]:
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: fd147a9f01c70dccbef40a31da78d34d4af2e6fda5f437c05b121d0be9cda428
  Private Key: 202ee6190a9f3aa40c3d4792a36d1c14bb6b3b9d7ff21788514809ab7c11d281
  Balance: 98 units
Wallet Bob:
  Public Key: 8e412a0238f1c4a0fd877bee6395e20ca9e1094ea4214f4c2e13ab5af9870e06
  Private Key: 423227a321216ed9ed326012e9053c3c9d2746e7d573dd7211a7a98995a57ded
  Balance: 55 units
Wallet Charlie:
  Public Key: 74db44b942863ec16933ef2d01237a3c91352d1f965901e59021265359cad2a9
  Private Key: 468e953f6241315da3aeac79ad4a9639604350cd362f37aa0e2af30b0c36c354
  Balance: 27 units
